## Commons( DataSet, Loader )

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from utils.utils import train_model, evaluate_model_with_cm, TiffDataset, base_transform,he_init_weights
from models.video_classifier import VideoClassifier, BandExpansion, ResBlock, FeedForward,init
import os
import numpy as np
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
source_data_dir = rf"C:\Users\전승표\서울과기대\JupyterNotebook\Capstone\git_folder\data_final\source_data"
label_file_path = rf"C:\Users\전승표\서울과기대\JupyterNotebook\Capstone\git_folder\data_final\label_data\species\label_mapping_sampled.csv"
test_filter = lambda x: x >= 1 and (x % 50) in {1, 6, 13, 18, 25, 30, 32, 37, 44, 49}

## Bands만 이용 ( 5월 , patch_size=3, resolution = 7.5m)

In [17]:
epochs = 30
bands = 10
patch_size = 9 # 모델 안에서 3*3으로 변경

train_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_int16"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: not test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)
val_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_int16"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


### CNN video classification

In [18]:
class BandVideoClassifier(nn.Module):
    def __init__(self, input_bands, stage_repeats, stage_channels, num_classes=6, dropout=0.3):
        super().__init__()
        
        # 초기 밴드 확장 (입력 밴드 수 -> 첫 번째 stage 채널 크기)
        self.initial_conv = BandExpansion(input_bands, stage_channels[0])
        
        self.stages = nn.ModuleList()
        in_channels = stage_channels[0]
        
        # 4개의 Stage 구성
        for stage_idx in range(4):
            # 기본 ResBlock 반복 적용
            blocks = [ResBlock(in_channels, dropout) for _ in range(stage_repeats[stage_idx])]
            self.stages.append(nn.Sequential(*blocks))
            
            # 마지막 Stage를 제외하고 밴드 확장 수행
            if stage_idx < 3:
                self.stages.append(BandExpansion(in_channels, stage_channels[stage_idx + 1]))
                in_channels = stage_channels[stage_idx + 1]
        
        # 글로벌 평균 풀링 적용 (공간 차원 제거)
        self.gap = nn.AdaptiveAvgPool3d(1)
        
        # 피드포워드 블록 2개 추가
        self.ff1 = FeedForward(in_channels, dropout=dropout)
        self.ff2 = FeedForward(in_channels, dropout=dropout)
        
        # 최종 분류기 (Linear 레이어)
        self.classifier = nn.Linear(in_channels, num_classes)
        
        # 가중치 초기화 적용
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Conv3d) or isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                init.constant_(m.bias, 0)
    
    def forward(self, x):
        batch, bands, time, h, w = x.size()
        x = x[:,:,4,:,:].squeeze(2) #32, 10, 9, 9
        #print(x.shape, x[0,0,:,:])
        #x = F.avg_pool2d(x, kernel_size=3,stride=3,padding=0) #32, 10, 3, 3 해상도 축소
        patches = x.unfold(2, 3, 3).unfold(3, 3, 3)
        # 패치를 평탄화하여 중간값 계산을 위한 형태로 변환
        patches_flat = patches.reshape(batch, bands, -1, 9)

        # 정렬하여 중간값(4번째 요소) 찾기
        sorted_patches, _ = torch.sort(patches_flat, dim=-1)
        median_values = sorted_patches[:, :, :, 4]

        # 중간값을 원래 크기의 텐서로 재구성
        new_h = (h - 3) // 3 + 1
        new_w = (w - 3) // 3 + 1
        x = median_values.reshape(batch,bands,-1 , new_h, new_w)
        #print(x.shape, x[0,0,0,:,:])
        #x = F.interpolate(x, size=(x.shape[2]*3 ,x.shape[3]*3), mode='nearest') #32, 10, 9, 9
        #x = x.unsqueeze(2) 
        #print(x.shape, x[0,0,:,:,:])
        x = self.initial_conv(x)  # 초기 밴드 확장 적용
        
        # Stage 반복 수행
        for stage in self.stages:
            x = stage(x)
        
        # 글로벌 평균 풀링 적용 후 차원 축소
        x = self.gap(x).view(x.size(0), -1)
        
        # 피드포워드 블록 두 개 통과
        x = self.ff1(x.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1))
        x = self.ff2(x)
        
        # 최종 분류기 적용
        x = self.classifier(x.squeeze(-1).squeeze(-1).squeeze(-1))
        return x
    
# 모델 설정
stage_repeats = [2, 2, 3, 2]  # 각 stage에서 ResBlock 반복 횟수
stage_channels = [16,24,35,48] # 각 stage의 채널 크기 [16,24,48,96] =>263,958
num_classes = 6  # 분류할 클래스 개수
bands = 10

# 모델 생성
model = BandVideoClassifier(bands, stage_repeats, stage_channels, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"파라미터 개수: {total_params}")


파라미터 개수: 264054


In [19]:
best_model_state, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=epochs)
os.makedirs("./checkpoints/ablation_study", exist_ok=True)
torch.save(best_model_state, f"./checkpoints/ablation_study/cnn_videoclassification_{bands}_month5_{patch_size}_{epochs}.pth")

Epoch 1/30 - Training:   0%|          | 0/5680 [00:00<?, ?it/s]

Epoch 1/30 - Validation: 100%|██████████| 1416/1416 [00:24<00:00, 56.97it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 


Epoch [1/30], Train Loss: 2.3290, Train Accuracy: 0.21, Train f1-score: 0.18 Val Loss: 1.8489, Val Accuracy: 0.11, Val f1-score: 0.03



Epoch 2/30 - Validation: 100%|██████████| 1416/1416 [00:24<00:00, 58.42it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 


Epoch [2/30], Train Loss: 1.8185, Train Accuracy: 0.28, Train f1-score: 0.19 Val Loss: 1.9134, Val Accuracy: 0.13, Val f1-score: 0.06



...


Epoch [28/30], Train Loss: 0.6343, Train Accuracy: 0.76, Train f1-score: 0.74 Val Loss: 1.1248, Val Accuracy: 0.70, Val f1-score: 0.72



Epoch 29/30 - Validation: 100%|██████████| 1416/1416 [00:23<00:00, 59.36it/s]



Epoch [29/30], Train Loss: 0.6282, Train Accuracy: 0.76, Train f1-score: 0.75 Val Loss: 1.0238, Val Accuracy: 0.66, Val f1-score: 0.67



Epoch 30/30 - Validation: 100%|██████████| 1416/1416 [00:24<00:00, 57.75it/s]



Epoch [30/30], Train Loss: 0.6206, Train Accuracy: 0.76, Train f1-score: 0.75 Val Loss: 0.9290, Val Accuracy: 0.69, Val f1-score: 0.71



In [20]:
del model,train_loader,val_loader,best_model_state
torch.cuda.empty_cache()

## Spatial만 사용( band=NIR, month=5, resolution=2.5m , patch_size=9 )

In [2]:
epochs = 30
bands = 10
patch_size = 3 # 모델 안에서 3*3으로 변경

train_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_int16"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: not test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)
val_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_int16"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


KeyboardInterrupt: 

In [ ]:
class SpatialVideoClassifier(nn.Module):
    def __init__(self, input_bands, stage_repeats, stage_channels, num_classes=6, dropout=0.3):
        super().__init__()
        
        # 초기 밴드 확장 (입력 밴드 수 -> 첫 번째 stage 채널 크기)
        self.initial_conv = BandExpansion(input_bands, stage_channels[0])
        
        self.stages = nn.ModuleList()
        in_channels = stage_channels[0]
        
        # 4개의 Stage 구성
        for stage_idx in range(4):
            # 기본 ResBlock 반복 적용
            blocks = [ResBlock(in_channels, dropout) for _ in range(stage_repeats[stage_idx])]
            self.stages.append(nn.Sequential(*blocks))
            
            # 마지막 Stage를 제외하고 밴드 확장 수행
            if stage_idx < 3:
                self.stages.append(BandExpansion(in_channels, stage_channels[stage_idx + 1]))
                in_channels = stage_channels[stage_idx + 1]
        
        # 글로벌 평균 풀링 적용 (공간 차원 제거)
        self.gap = nn.AdaptiveAvgPool3d(1)
        
        # 피드포워드 블록 2개 추가
        self.ff1 = FeedForward(in_channels, dropout=dropout)
        self.ff2 = FeedForward(in_channels, dropout=dropout)
        
        # 최종 분류기 (Linear 레이어)
        self.classifier = nn.Linear(in_channels, num_classes)
        
        # 가중치 초기화 적용
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Conv3d) or isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = x[:,3,4,:,:].unsqueeze(1).unsqueeze(2) #32, 1,1, 9, 9
        #print(x.shape)
        #print( x[0,0,0,:,:])
        x = self.initial_conv(x)  # 초기 밴드 확장 적용
        
        # Stage 반복 수행
        for stage in self.stages:
            x = stage(x)
        
        # 글로벌 평균 풀링 적용 후 차원 축소
        x = self.gap(x).view(x.size(0), -1)
        
        # 피드포워드 블록 두 개 통과
        x = self.ff1(x.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1))
        x = self.ff2(x)
        
        # 최종 분류기 적용
        x = self.classifier(x.squeeze(-1).squeeze(-1).squeeze(-1))
        return x
    
# 모델 설정
stage_repeats = [2, 2, 3, 2]  # 각 stage에서 ResBlock 반복 횟수
stage_channels = [16,24,35,48] # 각 stage의 채널 크기 [16,24,48,96] =>263,958
num_classes = 6  # 분류할 클래스 개수
bands = 1

# 모델 생성
model = SpatialVideoClassifier(bands, stage_repeats, stage_channels, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"파라미터 개수: {total_params}")


파라미터 개수: 263910


In [ ]:
best_model_state, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=epochs)
os.makedirs("./checkpoints/ablation_study", exist_ok=True)
torch.save(best_model_state, f"./checkpoints/ablation_study/cnn_videoclassification_NIR_month5_{patch_size}_2-5m_{epochs}.pth")
del model,train_loader,val_loader,best_model_state
torch.cuda.empty_cache()

Epoch 1/30 - Validation: 100%|██████████| 1420/1420 [00:25<00:00, 55.57it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 


Epoch [1/30], Train Loss: 2.2493, Train Accuracy: 0.20, Train f1-score: 0.17 Val Loss: 1.8393, Val Accuracy: 0.10, Val f1-score: 0.03



Epoch 2/30 - Validation: 100%|██████████| 1420/1420 [00:21<00:00, 64.57it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 


Epoch [2/30], Train Loss: 1.9233, Train Accuracy: 0.23, Train f1-score: 0.17 Val Loss: 1.8517, Val Accuracy: 0.11, Val f1-score: 0.04



Epoch 3/30 - Validation: 100%|██████████| 1420/1420 [00:48<00:00, 29.09it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

...


Epoch [28/30], Train Loss: 1.5403, Train Accuracy: 0.36, Train f1-score: 0.26 Val Loss: 1.5457, Val Accuracy: 0.34, Val f1-score: 0.25



Epoch 29/30 - Validation: 100%|██████████| 1420/1420 [00:21<00:00, 65.10it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control


Epoch [29/30], Train Loss: 1.5398, Train Accuracy: 0.35, Train f1-score: 0.26 Val Loss: 1.5520, Val Accuracy: 0.34, Val f1-score: 0.26



Epoch 30/30 - Validation: 100%|██████████| 1420/1420 [00:21<00:00, 65.04it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control


Epoch [30/30], Train Loss: 1.5394, Train Accuracy: 0.35, Train f1-score: 0.26 Val Loss: 1.5385, Val Accuracy: 0.34, Val f1-score: 0.25



## Time만 사용( band=NIR, resolution=7.5m , patch_size=3 )

In [3]:
epochs = 30
bands = 10
patch_size = 9 # 모델 안에서 3*3으로 변경

train_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_int16"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: not test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)
val_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_int16"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [20]:
class TimeVideoClassifier(nn.Module):
    def __init__(self, input_bands, stage_repeats, stage_channels, num_classes=6, dropout=0.3):
        super().__init__()
        
        # 초기 밴드 확장 (입력 밴드 수 -> 첫 번째 stage 채널 크기)
        self.initial_conv = BandExpansion(input_bands, stage_channels[0])
        
        self.stages = nn.ModuleList()
        in_channels = stage_channels[0]
        
        # 4개의 Stage 구성
        for stage_idx in range(4):
            # 기본 ResBlock 반복 적용
            blocks = [ResBlock(in_channels, dropout) for _ in range(stage_repeats[stage_idx])]
            self.stages.append(nn.Sequential(*blocks))
            
            # 마지막 Stage를 제외하고 밴드 확장 수행
            if stage_idx < 3:
                self.stages.append(BandExpansion(in_channels, stage_channels[stage_idx + 1]))
                in_channels = stage_channels[stage_idx + 1]
        
        # 글로벌 평균 풀링 적용 (공간 차원 제거)
        self.gap = nn.AdaptiveAvgPool3d(1)
        
        # 피드포워드 블록 2개 추가
        self.ff1 = FeedForward(in_channels, dropout=dropout)
        self.ff2 = FeedForward(in_channels, dropout=dropout)
        
        # 최종 분류기 (Linear 레이어)
        self.classifier = nn.Linear(in_channels, num_classes)
        
        # 가중치 초기화 적용
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Conv3d) or isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                init.constant_(m.bias, 0)
    
    def forward(self, x):
        batch, bands, time, h, w = x.size()
        x = x[:,3,:,:,:].unsqueeze(1) #32, 1,12, 9, 9
        #print(x.shape)
        #print(x.shape, x[0,0,:,:])
        #x = F.avg_pool2d(x, kernel_size=3,stride=3,padding=0) #32, 10, 3, 3 해상도 축소
        patches = x.unfold(2, 3, 3).unfold(3, 3, 3)
        # 패치를 평탄화하여 중간값 계산을 위한 형태로 변환
        patches_flat = patches.reshape(batch, time, -1, 9)

        # 정렬하여 중간값(4번째 요소) 찾기
        sorted_patches, _ = torch.sort(patches_flat, dim=-1)
        median_values = sorted_patches[:, :, :, 4]

        # 중간값을 원래 크기의 텐서로 재구성
        new_h = (h - 3) // 3 + 1
        new_w = (w - 3) // 3 + 1
        x = median_values.reshape(batch,-1 ,time, new_h, new_w)
        #print(x.shape)
        #print( x[0,0,0,:,:])
        
        x = self.initial_conv(x)  # 초기 밴드 확장 적용
        
        # Stage 반복 수행
        for stage in self.stages:
            x = stage(x)
        
        # 글로벌 평균 풀링 적용 후 차원 축소
        x = self.gap(x).view(x.size(0), -1)
        
        # 피드포워드 블록 두 개 통과
        x = self.ff1(x.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1))
        x = self.ff2(x)
        
        # 최종 분류기 적용
        x = self.classifier(x.squeeze(-1).squeeze(-1).squeeze(-1))
        return x
    
# 모델 설정
stage_repeats = [2, 2, 3, 2]  # 각 stage에서 ResBlock 반복 횟수
stage_channels = [16,24,35,48] # 각 stage의 채널 크기 [16,24,48,96] =>263,958
num_classes = 6  # 분류할 클래스 개수
bands = 1

# 모델 생성
model = TimeVideoClassifier(bands, stage_repeats, stage_channels, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"파라미터 개수: {total_params}")


파라미터 개수: 263910


In [21]:
best_model_state, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=epochs)
os.makedirs("./checkpoints/ablation_study", exist_ok=True)
torch.save(best_model_state, f"./checkpoints/ablation_study/cnn_videoclassification_NIR_all_{patch_size}_7-5m_{epochs}.pth")
del model,train_loader,val_loader,best_model_state
torch.cuda.empty_cache()

Epoch 1/30 - Validation: 100%|██████████| 1416/1416 [00:30<00:00, 45.79it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 


Epoch [1/30], Train Loss: 2.3602, Train Accuracy: 0.19, Train f1-score: 0.17 Val Loss: 1.8148, Val Accuracy: 0.19, Val f1-score: 0.05



Epoch 2/30 - Validation: 100%|██████████| 1416/1416 [00:23<00:00, 60.62it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 


Epoch [2/30], Train Loss: 1.9652, Train Accuracy: 0.21, Train f1-score: 0.17 Val Loss: 1.8389, Val Accuracy: 0.19, Val f1-score: 0.06



Epoch 3/30 - Validation: 100%|██████████| 1416/1416 [00:23<00:00, 59.84it/s]
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\전승표\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

...


Epoch [28/30], Train Loss: 0.8500, Train Accuracy: 0.66, Train f1-score: 0.64 Val Loss: 0.9489, Val Accuracy: 0.59, Val f1-score: 0.61



Epoch 29/30 - Validation: 100%|██████████| 1416/1416 [00:24<00:00, 58.68it/s]



Epoch [29/30], Train Loss: 0.8443, Train Accuracy: 0.67, Train f1-score: 0.65 Val Loss: 0.9065, Val Accuracy: 0.62, Val f1-score: 0.63



Epoch 30/30 - Validation: 100%|██████████| 1416/1416 [00:24<00:00, 58.86it/s]



Epoch [30/30], Train Loss: 0.8409, Train Accuracy: 0.67, Train f1-score: 0.65 Val Loss: 0.9190, Val Accuracy: 0.62, Val f1-score: 0.63

